In [ ]:
# !apt update
# !apt -y install ffmpeg

# Install this part from terminal using sudo for the password

In [1]:
import pandas as pd
import os
import re
# from tqdm.auto import tqdm
from tqdm.autonotebook import tqdm as notebook_tqdm
notebook_tqdm.pandas()

/tmp/ipykernel_18769/719634918.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [ ]:
cwd = os.getcwd()
cwd

In [ ]:
root_path = cwd.removesuffix('notebooks/jupyter_notebooks')
root_path

In [4]:
df = pd.read_csv(root_path + 'raw_data/training_data/asr.csv') #your dataset here
df.head(20)
df.groupby('lang').head(2)

,id,start,end,source,lang,transcription
0,0,1.680,3.810,5804.mp3,bak,sertifikat taratəp jœrœjœŋmœ ællæ hawmə
1,1,3.800,4.600,5804.mp3,bak,jaq
1574,1574,40.623,42.404,7378.mp3,evn,laŋilwərtawutt͡ʃəŋkiːtin
1575,1575,25.404,27.031,7379.mp3,evn,əŋkiːwuntəɣəttəantimati
5453,5453,1.418,3.522,11257.mp3,mhr,ukeməjmomnalənonal
5454,5454,2.963,4.532,11257.mp3,mhr,nalaʃkyleʃəlevetjodaʃ
8966,8966,0.000,1.593,14770.mp3,xas,dmgimsobie
8967,8967,1.593,2.390,14770.mp3,xas,tarzittə
84018,84018,0.000,5.304,89822.mp3,tat,ber pensioner telefon kujdɤrmɑkɕɤ bulɤp stɑnts...
84019,84019,0.000,3.936,89823.mp3,tat,ɑlɑr ber jɑktɑn ɡɤnɑ keʃeɡa oxʃɑp kyrenalar


In [3]:
list_lang = list(df['lang'].unique()) 
list_lang

['bak', 'evn', 'mhr', 'xas', 'tat', 'sah']

'xas' has around 75000 entries, so we skip it for the processing

In [31]:
list_lang = ['bak', 'evn', 'mhr', 'tat', 'sah']

In [4]:

new_dir = root_path + 'processed_data' #new directory for cut files
if os.path.exists(new_dir) is False:
    os.mkdir(new_dir)
else:
    print('folder already exists')
    
list_lang = list(df['lang'].unique())    
    
# Create sub-folders with respective lang codes    
    
for l in list_lang:
    
    new_dir_ext = new_dir + f'/{l}'
    
    if os.path.exists(new_dir_ext) is False:
        os.mkdir(new_dir_ext)
    else:
        print('folder already exists')

folder already exists
folder already exists
folder already exists
folder already exists
folder already exists
folder already exists
folder already exists


In [5]:
df["fpath"] = root_path + 'raw_data/training_data/' + df["source"].astype(str)
df

,id,start,end,source,lang,transcription,fpath
0,0,1.680,3.810,5804.mp3,bak,sertifikat taratəp jœrœjœŋmœ ællæ hawmə,../raw_data/training_data/5804.mp3
1,1,3.800,4.600,5804.mp3,bak,jaq,../raw_data/training_data/5804.mp3
2,2,5.060,5.680,5804.mp3,bak,ultər,../raw_data/training_data/5804.mp3
3,3,8.580,11.480,5804.mp3,bak,hin nindæj dokumenttar birðeŋ tip horarɣa kil...,../raw_data/training_data/5804.mp3
4,4,12.315,14.015,5804.mp3,bak,hin xatta dokument aləp barmaɣanəŋmə,../raw_data/training_data/5804.mp3
...,...,...,...,...,...,...,...
98541,98541,0.000,9.180,110217.mp3,sah,tyːn ajaŋːa tuɾumːut ol baɾan ihen at xajatɯn ...,../raw_data/training_data/110217.mp3
98542,98542,0.000,8.280,110218.mp3,sah,aɾaːha aɾɣaː dieki tuox eɾe olus ulaxan alɟʝaɾ...,../raw_data/training_data/110218.mp3
98543,98543,0.000,9.108,110219.mp3,sah,tyːny bɯha kulun kuɾduk øɾø møxsøn meneɾijeːɾi...,../raw_data/training_data/110219.mp3
98544,98544,0.000,7.920,110220.mp3,sah,ɯnaxsɯt uonːa soxsoloox ikːi aɾdɯɡaɾ kuːhuma o...,../raw_data/training_data/110220.mp3


In [6]:
def replacer(path):
    return path.replace(' ', '_')
df['fpath'] = df['fpath'].apply(replacer)
df = df.reset_index() #adding indexes(id)

In [30]:
!mkdir -p {root_path}/processed_data/ffmpeg_log

def cutter(row): # cutting files accroding to timecodes
    
    fpath, start, end, index, lang = row["fpath"], row["start"], row["end"], row["index"], row["lang"]
    
    if pd.isna(row.transcription) == False:    
        
        if pd.isna(row.start) or pd.isna(row.end):
            
            !ffmpeg -n -i {fpath} -ar 16000 \
            {root_path + 'processed_data/' + str(lang) + "/" + str(index)}.mp3 \
            2> {root_path}/processed_data/ffmpeg_log/{index}.log
        
        else:
        
            !ffmpeg -n -i {fpath} -ss {str(start)} -to {str(end)} -ar 16000 \
            {root_path + 'processed_data/' + str(lang) + "/" + str(index)}.mp3 \
            2> {root_path}/processed_data/ffmpeg_log/{index}.log

            
for l in list_lang:
    
    df_l = df[df['lang']==l]
    df_l.progress_apply(cutter, axis=1)
    
    df_l = df_l.dropna(subset=['transcription'])
    
    # making column for paths of cut files
    df_l['file_name'] = df_l['index'].apply(lambda x: str(x) + '.mp3')
    
    
    # cleaning transcriptions
    # df_l['transcription_clean'] = df_l['transcription'].apply(lambda x: x.strip('.«,').replace('=', '').replace(' ', '').replace('Ø', ' ')
    df_l['transcription_clean'] = df_l['transcription'].apply(lambda x: x.strip('.«,').replace('=', '').replace('Ø', ' ')) # clearing punctuation marks and spaces
    df_l['transcription_clean'] = df_l['transcription_clean'].apply(lambda x: re.sub('\(.+?\)', '', x))
   
    # filling empty strings
    df_l['transcription_clean'] = df_l['transcription_clean'].apply(lambda s: s if s else '-')
        
    # saving file
    df_l.to_csv(f'{root_path}/processed_data/{l}/asr_{l}.csv', index=False)
    
    # dropping columns which are not needed in the metadata file
    df_l.drop(columns=['fpath', 'id', 'start', 'end', 'index', 'source', 'lang'], inplace=True)
    
    df_l = df_l[['file_name', 'transcription']]
    df_l.to_csv(f'{root_path}/processed_data/{l}/metadata.csv', index=False)

  0%|          | 0/11970 [00:00<?, ?it/s]

  0%|          | 0/2558 [00:00<?, ?it/s]

Some analysis for Evenki language transcripts : we have 4 cases where the person speaks only Russian, so transcription is set to NaN.

In [13]:
df_evn = df[df['lang'] == 'evn']

num_mask = pd.to_numeric(df_evn['transcription'], errors='coerce').isnull()

df_evn.loc[num_mask, 'transcription']

print(df_evn.transcription[df_evn.transcription.isna()]) 

4244    NaN
4298    NaN
4591    NaN
5347    NaN
Name: transcription, dtype: object


Some analysis for Bashkir Language transcripts : we have cases where start and end of recording are set to NaN, this is for very short audio, i.e. there is no need to cut the audio --> in the preprocessing we don't cut and skip them

In [25]:
df_bak = df[df['lang'] == 'bak']

print(df_bak.start[df_bak.start.isna()]) 

7928   NaN
7929   NaN
7930   NaN
7931   NaN
7932   NaN
        ..
8961   NaN
8962   NaN
8963   NaN
8964   NaN
8965   NaN
Name: start, Length: 992, dtype: float64


In [27]:
num_mask = pd.to_numeric(df_bak['transcription'], errors='coerce').isnull()

df_bak.loc[num_mask, 'transcription']

print(df_bak.transcription[df_bak.transcription.isna()]) 

Series([], Name: transcription, dtype: object)


Some analysis for Tatar language

In [28]:
df_tat = df[df['lang'] == 'tat']

print(df_tat.start[df_tat.start.isna()]) 

Series([], Name: start, dtype: float64)


In [29]:
num_mask = pd.to_numeric(df_tat['transcription'], errors='coerce').isnull()

df_tat.loc[num_mask, 'transcription']

print(df_tat.transcription[df_tat.transcription.isna()]) 

Series([], Name: transcription, dtype: object)


After this audio cutting procedure also all the audio files are normalized to have 16Hz